# Our Task: Find similar company names

Now that we have completed our Python Primer, let's try to complete a real task, while at the same time trying to practice loops, iterations, and other Python functionality that we studied.

### Your high level task

You are given a list of names. You know that the same entity in the list has different representations. You want to find duplicate companies in the data.

As a concrete example, open the file under `data/restaurants.txt`. This contains a list of restaurant names, extracted from the [NYC Restaurant inspection data set](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/xx67-kt59/data) (available online). The Department of Health has been doing a decent, but not perfect, job in recording the company names. Therefore, the same restaurant appears under different names. **Your task is to find "almost duplicate" entries in order to decide whether they correspond to the same business.**


#### Matching Company Names

Quite often, in our data, we have entries represented as strings that refer to the same entity but have different string representations (e.g., McDonald's vs McDonalds vs McDonald‎). We want to write code that will help in the task of finding such similar entries in our data.

Our task can be broken in the following tasks:
* **Step 1**: Read the data from a file into a list of strings in memory. We have two data sets under the `data` folder: The list of unique restaurant names from the NYC Restaurant Inspection dataset (`data/restaurants.txt`). We need to write Python code that will read these files and return a list of strings that are the company names.
* **Step 2**: We will then need to figure out how to compare two strings and find their similarity. For that, we will write a function that takes as input **two** strings, and returns back their similarities. We will explore multiple ways of writing that function, using various library functions.
* **Step 3**: We will need to write a function that takes as input a company name, and returns back a list of matching company names, together with their similarity. Ideally, we would like to also sort the list based on the similarity (highest similarity metrics on top). As part of our learning process, we will use the _list comprehension_ approach to create the list. We will also use tuples as the elements of the list, so that the list contain elements such as `[("McDonalds", 0.88), ("McDonald's", 0.81),....]`.
* **Step 4**: In the final step, we will just perform the similarity computation across all companies in the dataset.

In [5]:
# first download a list of restaurants names in NYC 
!curl http://people.stern.nyu.edu/nwhite/DealingwithDataSpring2021/6-Pandas/data/restaurants.txt > restaurants.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7173k  100 7173k    0     0   713k      0  0:00:10  0:00:10 --:--:--  886k


### STEP 1: Read the list of names from a file and create a list of names

In [6]:
# STEP 1: Read the list of names from a file and create a list of names
filename = './restaurants.txt'

f = open(filename, 'r')
content = f.read()
f.close()
names = sorted(set(content.split('\n')))
names

['',
 '"1',
 '"1174 FULTON CUISINE',
 '"70KING',
 '"AIR BAR',
 '"AU BON PAIN',
 '"AUNTIE ANNE\'S PRETZELS',
 '"BAKED BY MELISSA',
 '"BECEC',
 '"BISOUS',
 '"BLENDED-UP JUICE',
 '"BOOM BOOM ROOM',
 '"BUFFALO WILD WINGS',
 '"BURGER KING',
 '"CADWALADER',
 '"CARVEL ICE CREAM',
 '"CHATIME',
 '"CHIC-FIL-A',
 '"CHICKEN/CELEBRITY CHEF',
 '"CHICKPEA',
 '"CHINA MIA EXOTIC ASIAN FOOD',
 '"D\'ANGELO\'S PIZZA',
 '"DELICIAS CUENCANAS BAR-RESTAURANT DELI',
 '"DESSERT CLUB',
 '"DUNKIN DONUTS',
 '"DUNKIN\' DONUTS',
 '"EMPLOYEE CAFETERIA',
 '"FISHERMAN\'S COVE',
 '"FLIP BIRD',
 '"FLOYD',
 '"FRIEND\'S FRIED CHICKEN',
 '"GARDEN MARKET',
 '"GROVE SMOOTHIE KING',
 '"HAAGEN- DAZS',
 '"HAMBURGER/CELEBRITY CHEF',
 '"HAMPTON CHUTNEY',
 '"HUMRO BHIM\'S CAFE',
 '"HYATT',
 '"IRIS TEA & BAKERY',
 '"KFC',
 '"LOVE CRAFT',
 '"MAMA VAL\'S FISH',
 '"NY CAROUSEL ENTERTAINMENT',
 '"NY FOOD COURT JAMAICA(CHINESE FOOD',
 '"NYSC',
 '"OAXACA TAQUERIA',
 '"ORGANIC FOOD',
 '"PIZZA HUT',
 '"RESTAURANT ASSOCIATES',
 '"ROLLS',
 '"

In [7]:
!pip install kora
from kora import console
console.start()

     |████████████████████████████████| 61kB 4.9MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
Console URL: https://teleconsole.com/s/fa3f8598a5d2d950a2c4c7f557ace31435d90e4d


### STEP 2: Implement the similarity function

### Computing the similarity between two strings

There are many ways that we can calculate the similarity between two strings. For our case, we will focus on a few similarity metrics that already have implementations in Python. 

##### Some commonly used similarity metrics

* [Sequence matching](https://docs.python.org/3.6/library/difflib.html) (part of standard Python) ([example](http://stackoverflow.com/questions/17388213/find-the-similarity-percent-between-two-strings))
* [Edit distance](https://en.wikipedia.org/wiki/Edit_distance) ([Python Jellyfish Library](https://github.com/jamesturk/jellyfish))
* [N-gram distance](http://pythonhosted.org/ngram/tutorial.html#comparing-and-searching-strings)


#### STEP 2.a: Let's figure out how we can install the libraries...

In [8]:
# Edit distance
!sudo pip3 install -U jellyfish

     |████████████████████████████████| 102kB 6.0MB/s 


In [9]:
# Ngram
!sudo pip3 install -U ngram

  Created wheel for ngram: filename=ngram-3.3.2-cp36-none-any.whl size=24682 sha256=7c35b8692f5682e4e2046265f6acd80957395fc7a3081846531fc038035ff2e6
  Stored in directory: /root/.cache/pip/wheels/7b/d5/5a/6abe3035c83a606376089accde30b021ffb26593f0d6609ef1
Successfully built ngram


#### STEP 2.b: Now let's test our similarity functions with various examples

Once we have installed the necessary libraries for our project, we proceed to `import` them and test the functions.

In [10]:
import jellyfish

##### Edit Distance

From Wikipedia:

The [edit distance](https://en.wikipedia.org/wiki/Edit_distance) _ is a way of quantifying how dissimilar two strings (e.g., words) are to one another by counting the minimum number of operations required to transform one string into the other._. 

The Levenshtein distance between "kitten" and "sitting" is 3. A minimal edit script that transforms the former into the latter is:

* kitten → sitten (substitution of "s" for "k")
* sitten → sittin (substitution of "i" for "e")
* sittin → sitting (insertion of "g" at the end).

In [11]:
jellyfish.levenshtein_distance('kitten', 'sitting')

3

Let's try a few more examples

In [12]:
jellyfish.levenshtein_distance('Starbucks', 'Starbacks')

1

In [13]:
jellyfish.levenshtein_distance('Starbucks', 'Starbuck')

1

In [14]:
jellyfish.levenshtein_distance('Starbucks', 'Wendys')

8

##### Demerau Levenshtein distance

The Demerau Levenshtein distance also allows for  transposition of two adjacent characters.

In [15]:
jellyfish.damerau_levenshtein_distance('Starbucks', 'Starbucsk')

1

In [16]:
jellyfish.levenshtein_distance('Starbucks', 'Starbucsk')

2

###### Jaro–Winkler distance

[Jaro–Winkler distance](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance) is a string metric for measuring the edit distance between two sequences. Informally, the **Jaro** distance between two words is the minimum number of single-character transpositions required to change one word into the other; the **Jaro–Winkler** distance  gives more favourable ratings to strings that match from the beginning.


In [17]:
jellyfish.jaro_winkler('Starbucks', 'Starbarbr')

0.8222222222222222

In [18]:
jellyfish.jaro_winkler('Starbucks', 'Milwbucks')

0.7037037037037037

##### Soundex

[Soundex](https://en.wikipedia.org/wiki/Soundex) is a phonetic algorithm for indexing names by sound, as pronounced in English. The goal is for homophones to be encoded to the same representation so that they can be matched despite minor differences in spelling. 

Using this algorithm, both "Robert" and "Rupert" return the same string "R163" while "Rubin" yields "R150". "Ashcraft" and "Ashcroft" both yield "A261". "Tymczak" yields "T522" not "T520" (the chars 'z' and 'k' in the name are coded as 2 twice since a vowel lies in between them). "Pfister" yields "P236" not "P123" (the first two letters have the same number and are coded once as 'P').

In [19]:
jellyfish.soundex('Robert')

'R163'

In [20]:
jellyfish.soundex('Rupert')

'R163'

In [21]:
jellyfish.soundex('Ashcroft')

'A261'

In [22]:
jellyfish.soundex('Ashcraft')

'A261'

#### Ngrams

With the n-gram similarity score, we split the word into sequences of n consecutive characters (n-grams) and then compare the sets of n-grams between the two words. For example, the name "Panos" has the following 2-grams: "Pa", "an", "no", "os". (We can also add "#P" and "s#" if we want to capture the prefix and suffixes.) Strings that share a large number of q-grams are often similar.

In [23]:
import ngram

In [24]:
ngram.NGram.compare('Ipeirotis','Iperotis',N=2)

0.7272727272727273

In [25]:
ngram.NGram.compare('New York University','New York Universty',N=2)

0.8571428571428571

#### Task 1: Create a function that takes as input two strings and returns their similarity

Given the experience with the metrics above, we would like to create a function that takes as input a string and returns their similarity. Our key requirement is for the similarity metric to be between 0 and 1, with 0 meaning no similarity, and 1 corresponding to identical strings. Some of the similarity functions above would fit right in, others will need some work. 

In [26]:
# For n-gram similarity it is very simple, we just return the result
def computeSimilarity(str1, str2):
    return ngram.NGram.compare(str1,str2,N=2)

In [27]:
computeSimilarity('New York University','New York')

0.38095238095238093

In [28]:
# For edit distance, we need to normalize. 
# Edit distance is always the maximum of the length of the two strings
# (can you figure out why?) so we can use that to normalize the distance
def computeSimilarity(str1, str2):
    dist = jellyfish.levenshtein_distance(str1, str2)
    max_dist = max(len(str1), len(str2))
    similarity = 1 - dist/max_dist
    return similarity

In [29]:
computeSimilarity('New York University','New York')

0.42105263157894735

#### Task 2: Modify the functions above to allow for various similarity calculation methods.

We will now up our game, and return back the results of the comparison using multiple methods at once. 

In [30]:
def computeSimilarity_2gram(str1, str2):
    return ngram.NGram.compare(str1,str2,N=2)

def computeSimilarity_3gram(str1, str2):
    return ngram.NGram.compare(str1,str2,N=3)

def computeSimilarity_edit_distance(str1, str2):
    dist = jellyfish.levenshtein_distance(str1, str2)
    max_dist = max(len(str1), len(str2))
    similarity = 1 - dist/max_dist
    return similarity

def computeSimilarity_soundex(str1, str2):
    s1 = jellyfish.soundex(str1)
    s2 = jellyfish.soundex(str2)
    if (s1==s2):
        return 1
    else:
        return 0

In [25]:
def computeSimilarity(str1, str2, method):
    if method == '2-gram':
        return computeSimilarity_2gram(str1, str2)
    elif method == '3-gram':
        return computeSimilarity_3gram(str1, str2)
    elif method == 'levenshtein':
        return computeSimilarity_edit_distance(str1, str2)
    elif method == 'soundex':
        return computeSimilarity_soundex(str1, str2)
    
computeSimilarity('New York University','New York', method='3-gram')

0.34782608695652173

In [26]:
# This is a bit more advanced, conceptually.
# Instead of selecting a method to use, we use all available methods to 
# do our comparison, and we return a dictionary with the results
# This approach would be useful when dealing with the problem 
# as a classification problem (match vs no match) and we want to
# use the distance values as features
def computeSimilarity_all(str1, str2):
    result = {
        '2-gram': computeSimilarity_2gram(str1, str2),
        '3-gram': computeSimilarity_3gram(str1, str2),
        'levenshtein': computeSimilarity_edit_distance(str1, str2),
        'soundex': computeSimilarity_soundex(str1, str2)
    }
    
    return result

computeSimilarity_all('New York University','New York Universty')

{'2-gram': 0.8571428571428571,
 '3-gram': 0.782608695652174,
 'levenshtein': 0.9473684210526316,
 'soundex': 1}

### STEP 3: Compute similarity of a company against a list of company names

We now create a function that accepts a company name and a list of companies, and computes their similarity. This part will get us to exercise our for-loops, and also illustrate how we can use lists and tuples.

**Sorting a list of tuples**:_This part is a little bit advanced for now, so I will just give the code below. (Solution taken from http://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples). Here is a small example below, which we will reuse in our function:_

In [27]:
data = [("Panos",0.5), ("Peter",0.6), ("Pan", 0.4)]
data.sort(key=lambda tupl: tupl[1], reverse=True)  # sorts in place, in descending order, based on the second element of the tuple
print(data)

[('Peter', 0.6), ('Panos', 0.5), ('Pan', 0.4)]


In [28]:
# STEP 3: We now create a function that accepts a company name
# and a list of companies, and computes their similarity
# We have a 'top' parameter (by default set to be 5)
# that restricts the results to only the most similar 
# string pairs. We also define a parameter "method" that defines 
# what is the similarity method that we want to use. We also define a 
# similarity threshold for keeping only results with sufficient similarity

def queryData(query, companyList, top = 5, method = '2-gram', sim_threshold = 0.5):
    # Initialize the list
    results = []
    # Go over all the companies in the data set....
    for company in companyList:
        # Compute the similarity of each company against the "query" variable
        sim = computeSimilarity(query, company, method)
        # We keep only results with sufficient similarity and we do not keep identical names
        if sim > sim_threshold and query!=company:
            # We store the similar company name, and the value of their similarity
            results.append( (company, sim) )
    # With the command below we sort the results, and put the most similar results in the top
    results.sort(key=lambda tupl: tupl[1], reverse=True)
    # We return only the first "threshold" results
    return results[:top]
      


In [29]:
r = queryData("MCDONALDS", names)
r

NameError: ignored

### Step 4: Perform the similarity computation across all companies in the dataset.

In [ ]:
# STEP 4: We are almost done. We now just go through all the companies in the list
# and we call the companySimilarity function that computes the similar company names
# for all the companies in the list. We store the results in a dictionary, with the 
# key being the company name, and the value being a "list of tuples" with the 
# similar company names and the corresponding similarity value.


In [ ]:
i = 0
for companyName in names:
    matches = queryData(companyName, names)
    for match, sim in matches:
        print(companyName, "\t", match, "\t", sim)
        i = i + 1
    if i>10: 
        break # We stop after 10 matches 